# Solar Flux for the 21-cm Line
Written by Austin Lake

Saddleback College, Mission Viejo

## Introduction
This Jupyter Notebook calculates the expected solar flux at the 21-cm line, as observered from Saddleback College's 2-meter radio telescope

## Method
First, the radiance of the Sun at an estimated temperature of 5778 K, is dervied used Planck's Law of blackbody radiation. This derivation outputs units of Watts per meter-squared per sterradian [W/m^2/sr]. Because the Sun lies within the FOV of the telescope, the solid angle [sr] is calcuated from a point behind the radio telescope that propagates outward toward the Sun. The center axis of propagation passes through the both the center of the dish and the center of the Sun. Additionally, the outside edges of this solid angle form a tangent line on the edge of the dish and the meridian of the Sun that is perpendicular to the axis of propagation. The radiance from the blackbody model is then multiplied by the solid angle and the telescope's aperature area [m^2] to get the flux [W]. This is then converted to fit a logrithimic scale, resulting in units of decibel milliwatt [dBm].

In [1]:
# dependencies
from astropy import units as u
from astropy.modeling.physical_models import BlackBody
from astropy.constants import au, R_sun
import numpy as np

## Solid Angle Relationship
![](solidAngle.png)

### Derving the Solid Angle of Our Dish and the Sun
$\Omega=\frac{A}{r^2}, 0\leq\Omega\leq 4\pi, r=\mathrm{radial\space distance}$

$\Omega=\frac{A_{dish}}{X^2}, A_{dish}=\mathrm{apeture}, X=\mathrm{radial\space distance\space to\space dish}$

$\Omega=\frac{A_{sun}}{(au + X)^2}, A_{sun}=\mathrm{hemisphere\space of\space Sun\space facing\space dish}$

$\frac{X^2}{A_{dish}}=\frac{(au+X)^2}{A_{sun}}$

$\frac{X^2}{A_{dish}}=\frac{\mathrm{(au)^2+2au(X)+X^2}}{\mathrm{A_{sun}}}$

$(\frac{1}{A_{dish}}-\frac{1}{A_{sun}})x^2-\frac{2au}{A_{sun}}X-\frac{(au)^2}{A_{sun}}=0$

$a=\frac{1}{A_{dish}}-\frac{1}{A_{sun}}=\frac{1}{2\pi (D_{dish})^2}-\frac{1}{2\pi ((r_{sun})/2)^2}=\frac{1}{\pi ((4.5\space m)/2)^2}-\frac{1}{2\pi (695700000.0\space m)^2}=0.06287602690050187$

$b=-\frac{2au}{A_{sun}}=\frac{2au}{2\pi (r_{sun})^2}=\frac{149597870700.0\space m}{\pi (695700000.0\space m)^2}=-1.967711973475555E-07$

$c=-\frac{(au)^2}{A_{sun}}=\frac{(au)^2}{2\pi (r_{sun})^2}=\frac{(149597870700.0\space m)^2}{2\pi (695700000.0\space m)^2}=-14718.276069141897$

$X=\frac{-b+\sqrt{b^2-4ac}}{a^2}=\frac{-(-1.967711973475555E-07)+\sqrt{(-1.967711973475555E-07)^2-4(0.06287602690050187)(-14718.276069141897)}}{(0.06287602690050187)^2}=\mathrm{483.82235182348757\space m}$

$\Omega=\frac{A_{dish}}{X^2}=\frac{\pi (D_{dish}/2)^2}{X^2}=\mathrm{\frac{\pi (4.5\space m/2)^2}{(483.82235182348757\space m)^2}=0.00013588547880587752\space sr}$




In [2]:
# calculate solid angle
dishDiameter = 4.5*u.m # diameter of telescope (m)
dishArea = (np.pi*(dishDiameter/2)**2) # area of the telescope's dish (m^2)
solarArea = 2*np.pi*R_sun**2 # surface area of solar hemisphere facing dish
a = ((1/dishArea)-(1/solarArea)).value # second order coeffecient
b = -(2*au/solarArea).value # first order coefficent
c = -(au**2/solarArea).value # zeroeth order coefficent
x = ((0-b+np.sqrt(b**2-4*a*c))/(2*a))*u.m # distance from dish to origin of solid angle (m)
solidAngle = (dishArea/(x**2))*u.sr # solid angle (m)

## Black Body Curve
![](blackbody_curves.png)

### Planck's Law of Blackbody Radiation

$\mathrm{radiance:\space}B(v,t)=\frac{2hv^3}{c^2}\cdot\frac{1}{\frac{hv}{e^{k_BT}}-1}$

$\mathrm{flux\space density = radiance\cdot solid\space angle}$

$\mathrm{spectral\space flux\space density=\frac{flux\space density}{frequency}}$

$\mathrm{1\space W/m^2/Hz=10^26\space Jy=10^24\space sfu}$

$\mathrm{flux= flux\space density\cdot apeture}$

$\mathrm{1\space W = 0\space dBm\space (logarithmic\space scale)}$

In [3]:
H1 = 1420405751.768*u.Hz # Hz
temperature = 5778*u.K # temperature(K)
scale = 1.0*u.W/u.m**2/u.sr # blackbody scale factor (W/m^2/sr)
solarModel = BlackBody(temperature=temperature,scale=scale) # black body model
radiance = solarModel(H1) # solar radiance (W/m^2/sr)
fluxDensity = radiance*solidAngle # flux density (W/m^2)
spectralFluxDensity = fluxDensity/H1 # spectral flux density (W/m^2/Hz)
jansky = spectralFluxDensity.to(u.Jy) # spectral flux density (Jy)
sfu = jansky.value*10e4 # spectral flux density (sfu)
flux = fluxDensity*dishArea # flux (W)
flux_dBm = u.Decibel(flux, u.DecibelUnit(u.mW)) # solar flux (dBm)

In [4]:
# display 
print(f'Solid angle (sr):\t\t\t\t\t\t{solidAngle}')
print(f'Flux (W):\t\t\t\t\t\t\t{flux}')
print(f'Flux (dBm):\t\t\t\t\t\t\t{flux_dBm}')
print(f'Flux Density (W/m^2):\t\t\t\t{fluxDensity}')
print(f'Spectral Flux Desnity (W/m^2/Hz):\t{spectralFluxDensity}')
print(f'Spectral Flux Density (Jy):\t\t\t{jansky}')
print(f'Spectral Flux Density (sfu):\t\t\t{sfu} sfu')


Solid angle (sr):						0.00013588547880587752 sr
Flux (W):							1.0994413652495469e-11 W
Flux (dBm):							-79.588279272196 dB(mW)
Flux Density (W/m^2):				6.912850485695501e-13 W / m2
Spectral Flux Desnity (W/m^2/Hz):	4.866813920663849e-22 W / (Hz m2)
Spectral Flux Density (Jy):			48668.13920663848 Jy
Spectral Flux Density (sfu):			4866813920.663848 sfu
